In [1]:
import sqlite3
import functions
import importlib # For reloading the functions.py module everytime I want to add something to it
importlib.reload(functions)
from functions import *
from math import sqrt, pow

conn = sqlite3.connect('elections.sqlite') # Connects to the sqlite database
c = conn.cursor()

print("Connection Established")

Connection Established


In [8]:
# Adds all the data we had in milestone 2 to the database

execute_SQL(conn, c, 'createAll.sql') # Creates the tables
execute_SQL(conn, c, 'loadAll.sql') # Loads all data

createAll.sql Successful Executed
loadAll.sql Successful Executed


# A. Activities:
1. a clerk creating a new ballot
2. a folk registering to vote for a ballot
3. a clerk modifying the availability period of a current ballot
4. a voter casting a ballot while confirming a valid voting registration by a monitor
    Use a SQL transaction to enture ACID properties (serializability) in the face of other concurrent oeprations or failures
5. a staff removing a folk (and all their associated information)

In [9]:
# 1.a. a clerk creating a new ballot

# ballot_ID = 4 # Leave this commented if you just want to use the next available ballot_ID (primary key)

# Uncomment the lines if you want to input your own values
# short_name = str(input("short_name: "))
# question_text = str(input("question_text: "))
# availability_ID = int(input("availability_ID: "))


# Predetermined values (feel free to change if you want)
short_name = "GH78"
question_text = "Should the country invest more into secondary education?"
availability_ID = 4

# If it asks you for the start_time and end_time, just use these values:
#   - 2023-10-28 08:00:00
#   - 2023-11-08 20:00:00


# Creates the new ballot
create_new_Ballots(conn, c, short_name, question_text, availability_ID)

New availability_ID detected. Please input data for the ballot's availability.


In [10]:
# 2.a. a folk registering to vote for a ballot

# voting_reg_ID = 25 # Leave this commented if you just want to use the next available voting_reg_ID (primary key)

# Uncomment the lines if you want to input your own values
# identifier = int(input("identifier: "))
# voting_date = str(input("voting_date: "))
# ballot_ID = int(input("ballot_ID: "))
# voting_center_ID = int(input("voting_center_ID: "))

# Predetermined values (feel free to change if you want)
identifier = 1
voting_date = "2023-10-28"
ballot_ID = 4
voting_center_ID = 1

# Creates the new voting registry
create_new_Voting_Registry(conn, c, identifier, voting_date, ballot_ID, voting_center_ID)

In [11]:
# 3. a clerk modifying the availability period of a current ballot

# Uncomment the lines if you want to input your own values
# ballot_ID = int(input("ballot_ID: "))
# start_time = str(input("start_time: "))
# end_time = str(input("end_time: "))

# Predetermined values (feel free to change if you want)
ballot_ID = 4
start_time = "2023-10-28 08:00:00"
end_time = "2023-11-09 20:00:00"

# Modifies the availability period of a current ballot
modify_ballot_availability(conn, c, ballot_ID, start_time, end_time)

In [12]:
# 4. a voter casting a ballot while confirming a valid voting registration by a monitor
#    Use a SQL transaction to enture ACID properties (serializability) in the face of other concurrent oeprations or failures

# cast_vote_ID = 19 # Leave this commented if you just want to use the next available cast_vote_ID (primary key)

# Uncomment the lines if you want to input your own values
# voter_ID = int(input("voter_ID: "))
# voting_reg_ID = int(input("voting_reg_ID: "))
# ballot_ID = int(input("ballot_ID: "))
# monitor_ID = int(input("monitor_ID: "))
# vote_time = str(input("vote_time"))
# vote = str(input("vote: "))

# Predetermined values (feel free to change if you want)
voter_ID = 1
voting_reg_ID = 25
ballot_ID = 4 # The latest ballot added
monitor_ID = 14
vote_time = "2023-10-28 10:00:00"
vote = "NO"

# Creates the new cast ballot
create_new_Cast_Votes(conn, c, voter_ID, voting_reg_ID, ballot_ID, monitor_ID, vote_time, vote)
# You can try running the command above again with the same voting_reg_ID. It'll tell you that the voting_reg_ID has already casted a vote (meaning that it works as intended).

In [13]:
# 5. a staff removing a folk (and all their associated information)

# Uncomment the lines if you want to input your own values
# personal_ID = int(input("personal_ID: "))

# Predetermined values (feel free to change if you want)
personal_ID = 12

# Removes the folk and all its info
remove_folk(conn, c, personal_ID)

name_ID = 12
emails_ID = 12


# B. Queries/Reports:
1. List the name, city and email (any single email suffices) of all folks.
2. List the city, state, and the number of residences of each city in Wonderland (skip cities with no residents) in decreasing order of number of residents.
3. List each state together with its number of currently inhabited places (include states with no inhabited places) in increasing alphabetical order.
4. Find the distinct identifiers of folks registered at a given voting center within a given time period.
5. Find for a given month, the number of unique registrations at any voting center which is within 5 miles from the center of Megapolis, except for voting centers in a given (exclusion) list of voting centers.
6. Find the most popular voting center(s) (in terms of total number of registrations) in a given time period among those in a given city.
7. Find the unique folk that have valid registrations with every voting center on a given state.
8. Find folks that registered at a voting center that is farther away than the voting center closest to their residence (break ties alphabetically by center's acronym).
9. Write a SQL function that returns the acronym of the voting center closest to the residence of a given folk among those whose operating period(s) contain a given date (return NULL if no such center exists; break ties at alphabetically by acronym).
10. For a given ballot, construct a cross-tabulation of voting centers (acronym) as rows, unique ballot answers (options) as columns, and cells indicating number of cast votes at the row's center with the column's option.

In [14]:
# 1. List the name, city and email (any single email suffices) of all folks.

command = """
SELECT first_name AS 'First Name', last_name AS 'Last Name', city AS 'City', email AS 'Email' FROM Folks
JOIN Folks_Name ON Folks.name_ID = Folks_Name.name_ID
JOIN Folks_Emails ON Folks.emails_ID = Folks_Emails.emails_ID
JOIN (
    SELECT * FROM Places
    JOIN Places_Residences ON Places.place_ID = Places_Residences.place_ID
    JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
)
GROUP BY Folks.personal_ID;
"""
column_names = ["First Name", "Last Name", "City", "Email"]

print_table(conn, c, column_names, command, bold_col_names=True)

FIRST NAME        LAST NAME        CITY        EMAIL        
Arturo            Washington       Chandal     HT3701@gmail.com
Megan             Norman           Chandal     HQ6413@gmail.com
Jody              Marsh            Chandal     TU9080@gmail.com
Micheal           Weiss            Chandal     PE0814@gmail.com
Rosario           Ho               Chandal     QB7606@gmail.com
Elizabeth         Ayers            Chandal     GM9365@gmail.com
Mayra             Baxter           Chandal     BL5613@gmail.com
Sonia             Drake            Chandal     MU9077@gmail.com
Chelsea           Hinton           Chandal     QW8964@gmail.com
Corey             Henderson        Chandal     BQ9542@gmail.com
Jenny             Lake             Chandal     FM3227@gmail.com



In [15]:
# 2. List the city, state, and the number of residences of each city in Wonderland (skip cities with no residents) in decreasing order of number of residents.

command = """
SELECT city AS 'City', state AS 'State', COUNT(city) AS 'Number of Residences in State' FROM Places
JOIN Places_Residences ON Places.place_ID = Places_Residences.place_ID
JOIN Folks ON Folks.residence_ID = Places_Residences.place_ID
JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
ORDER BY COUNT(city) DESC;
"""
column_names = ["City", "State", "Number of Residences in State"]

print_table(conn, c, column_names, command, bold_col_names=True)

CITY        STATE        NUMBER OF RESIDENCES IN STATE        
Chandal     Samarson     11                                   



In [16]:
# 3. List each state together with its number of currently inhabited places (include states with no inhabited places) in increasing alphabetical order.

command = """
SELECT state AS 'State', COUNT(Folks.residence_ID) AS 'Num Residences' FROM Places
LEFT JOIN Folks ON Folks.residence_ID = Places.place_ID
JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
GROUP BY Places.address_ID
ORDER BY State
"""
column_names = ["State", "Num Residences"]

print_table(conn, c, column_names, command, bold_col_names=True)

STATE        NUM RESIDENCES        
Livalla      0                     
Samarson     11                    



In [17]:
# 4. Find the distinct identifiers of folks registered at a given voting center within a given time period.

# Predetermined values (feel free to change if you want)
start_time = "2023-10-28"
end_time = "2023-11-02"
voting_center_ID = 1

command = f"""
SELECT DISTINCT identifier FROM ( 
    SELECT identifier, voting_date, first_name, last_name, voting_reg_ID, Places_Voting_Center.acronym FROM Voting_Registry
    JOIN Places_Voting_Center ON Voting_Registry.voting_center_ID = Places_Voting_Center.place_ID
    JOIN Folks ON Folks.personal_ID = Voting_Registry.identifier
    JOIN Folks_Name ON Folks.name_ID = Folks_Name.name_ID
    WHERE (voting_date BETWEEN '{start_time}' AND '{end_time}') AND (Places_Voting_Center.place_ID = {voting_center_ID})
    ORDER BY voting_date
)
"""
column_names = ["Identifier"]

print_table(conn, c, column_names, command, bold_col_names=True)

IDENTIFIER        
1                 
4                 



In [18]:
# 5. Find for a given month, the number of unique registrations at any voting center which is within 5 miles from the center of Megapolis, except for voting centers in a given (exclusion) list of voting centers.

# IMPORTANT NOTE: For some reason, I cannot use the SQRT() nor POWER(). Instead, I'll just manually add the distances for each voting center

# Predetermined values (feel free to change if you want)
month = "11"
excluded_voting_center_IDs = [2]
excluded_voting_center_IDs = format_tuple(excluded_voting_center_IDs)

command = f"""
SELECT Places.place_ID, x_coord, y_coord FROM Places
JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
JOIN Places_Coordinates ON Places.coordinates_ID = Places_Coordinates.coordinates_ID
"""
column_names = []

# print_table(conn, c, column_names, command, bold_col_names=True)

query_1 = get_query(conn, c, command)
nearby_voting_centers = [] # List of voting center IDs that are near Megapolis

# Add the distance from the origin on each voting center
for i in range(len(query_1)):
    query_1[i] = list(query_1[i])
    query_1[i].append(sqrt(pow(query_1[i][1], 2) + pow(query_1[i][2], 2)))
    query_1[i] = tuple(query_1[i])

    if query_1[i][3] <= 5:
        nearby_voting_centers.append(query_1[i][0])

nearby_voting_centers = format_tuple(nearby_voting_centers)




command = f"""
SELECT voting_center_ID, COUNT(voting_center_ID) FROM Voting_Registry
WHERE strftime('%m', voting_date) = '{month}' AND voting_center_ID IN {nearby_voting_centers} AND voting_center_ID NOT IN {excluded_voting_center_IDs}
GROUP BY voting_center_ID
"""
column_names = ["Voting_Center_ID", f"Num Unique Registrations @ Month #{month}"]

print_table(conn, c, column_names, command, bold_col_names=True)

VOTING_CENTER_ID        NUM UNIQUE REGISTRATIONS @ MONTH #11        
1                       4                                           
3                       4                                           



In [19]:
# 6. Find the most popular voting center(s) (in terms of total number of registrations) in a given time period among those in a given city.

# Predetermined values (feel free to change if you want)
start_time = "2023-10-28"
end_time = "2023-11-02"
city = "Watestone"

command = f"""
SELECT voting_center_ID, COUNT(voting_center_ID) FROM Voting_Registry
JOIN (
    SELECT Places.place_ID FROM Places
    JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
    JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
    WHERE city = '{city}'
) AS subquery ON Voting_Registry.voting_center_ID = subquery.place_ID
WHERE voting_date BETWEEN '{start_time}' AND '{end_time}'
GROUP BY voting_center_ID
ORDER BY COUNT(voting_center_ID) DESC
"""

# command = f"""
# SELECT Places.place_ID FROM Places
# JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
# JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
# WHERE city = '{city}'
# """
column_names = ["Voting_Center_ID", f"Num Voting Registrations Between '{start_time}' - '{end_time}'"]

print_table(conn, c, column_names, command, bold_col_names=True)

VOTING_CENTER_ID        NUM VOTING REGISTRATIONS BETWEEN '2023-10-28' - '2023-11-02'        
1                       5                                                                   
3                       4                                                                   
2                       4                                                                   



In [20]:
# 7. Find the unique folk that have valid registrations with every voting center on a given state.

# As of right now, there probably isn't a single folk that's registered for vote for every voting center. Let's try to get someone to register for every voting center.
c.execute(f"""
            INSERT INTO Voting_Registry
            VALUES ({get_next_ID(conn, c, 'Voting_Registry')}, 1, '2023-10-28', 4, 2);
""")
c.execute(f"""
            INSERT INTO Voting_Registry
            VALUES ({get_next_ID(conn, c, 'Voting_Registry')}, 1, '2023-10-28', 4, 3);
""")
conn.commit()



state = "Livalla"

# not gonna lie, this query involved me using all my braincells Q_Q (but it works me thinks)
command = f"""
SELECT subquery2.identifier FROM (
    SELECT DISTINCT identifier, subquery.place_ID, Num_Voting_Centers FROM Voting_Registry
    JOIN (
        SELECT Places_Voting_Center.place_ID, count.Num_Voting_Centers FROM Places
        JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
        JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
        CROSS JOIN (
            SELECT COUNT(*) AS 'Num_Voting_Centers' FROM Places
            JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
            JOIN Places_Address ON Places.address_ID = Places_Address.address_ID
            WHERE state = '{state}'
        ) AS count
        WHERE state = '{state}'
    ) AS subquery ON Voting_Registry.voting_center_ID = subquery.place_ID
    ORDER BY identifier
) AS subquery2
GROUP BY identifier
HAVING COUNT(subquery2.identifier) = Num_Voting_Centers
"""

print_table(conn, c, ["Identifier"], command, bold_col_names=True)

IDENTIFIER        
1                 



In [21]:
# 8. Find folks that registered at a voting center that is farther away than the voting center closest to their residence (break ties alphabetically by center's acronym).

# We first want info about which voting center is closest to each residence
command = """
SELECT Places.place_ID, role, x_coord, y_coord, acronym FROM Places
JOIN Places_Coordinates ON Places.coordinates_ID = Places_Coordinates.coordinates_ID
LEFT JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
"""
places_data = get_query(conn, c, command)

voting_centers = {}
residences = {}

for data in places_data:
    # print(data)

    if data[1] == "Residences":
        residences[data[0]] = {
            'x_coord': data[2],
            'y_coord': data[3]
        }
    elif data[1] == "Voting Center":
        voting_centers[data[0]] = {
            'x_coord': data[2],
            'y_coord': data[3],
            'acronym': data[4]
        }

for residence_place_ID in residences:
    started = False
    
    curr_closest_voting_center_distance = 0
    curr_closest_voting_center_address_ID = -1

    # Finds the closest voting center to each residence (creates a mapping of some sorts)
    for voting_center_place_ID in voting_centers:
        curr_distance = sqrt(pow(residences[residence_place_ID]['x_coord'] - voting_centers[voting_center_place_ID]['x_coord'], 2) + pow(residences[residence_place_ID]['y_coord'] - voting_centers[voting_center_place_ID]['y_coord'], 2))

        if started == False:
            curr_closest_voting_center_distance = curr_distance
            curr_closest_voting_center_address_ID = voting_center_place_ID
            started = True
        elif curr_distance < curr_closest_voting_center_distance:
            curr_closest_voting_center_distance = curr_distance
            curr_closest_voting_center_address_ID = voting_center_place_ID
        
        # break ties alphabetically by center's acronym
        elif curr_distance == curr_closest_voting_center_distance and voting_centers[voting_center_place_ID]['acronym'] < voting_centers[curr_closest_voting_center_address_ID]['acronym']:
            curr_closest_voting_center_distance = curr_distance
            curr_closest_voting_center_address_ID = voting_center_place_ID
    
    residences[residence_place_ID]['closest_voting_center_ID'] = curr_closest_voting_center_address_ID



# Creates a temporary table to make our queries easier
# The table creates a mapping where the key is the residence ID and the value is the ID of the closest voting center
command = """
CREATE TABLE Temp_Closest_Data (
    residence_ID INT,
    voting_center_ID INT
)
"""

c.execute(command)
conn.commit()



# Populates data from the residences dictionary
for place_ID in residences:
    command = f"""
    INSERT INTO Temp_Closest_Data
    VALUES ({place_ID}, {residences[place_ID]['closest_voting_center_ID']})
    """

    c.execute(command)
    conn.commit()


# The actual query for this section
command = f"""
SELECT DISTINCT Voting_Registry.identifier, Folks_Name.first_name, Folks_Name.last_name FROM Voting_Registry
JOIN Folks ON Voting_Registry.identifier = Folks.personal_ID
JOIN Temp_Closest_Data ON Temp_Closest_Data.residence_ID = Folks.residence_ID AND Temp_Closest_Data.voting_center_ID != Voting_Registry.voting_center_ID
JOIN Places_Voting_Center ON Places_Voting_Center.place_ID = Voting_Registry.voting_center_ID
JOIN Folks_Name ON Folks.name_ID = Folks_Name.name_ID
ORDER BY Voting_Registry.identifier
"""

print_table(conn, c, ["Personal_ID", "First Name", "Last Name"], command, bold_col_names=True)



# Drops the temporary table we made
command = """
DROP TABLE IF EXISTS TEMP_Closest_Data; 
"""

c.execute(command)
conn.commit()

PERSONAL_ID        FIRST NAME        LAST NAME        
1                  Arturo            Washington       
2                  Megan             Norman           
3                  Jody              Marsh            
5                  Rosario           Ho               
6                  Elizabeth         Ayers            
8                  Sonia             Drake            
9                  Chelsea           Hinton           
11                 Jenny             Lake             



In [22]:
# 9. Write a SQL function that returns the acronym of the voting center closest to the residence of a given folk among those whose operating period(s) contain a given date (return NULL if no such center exists; break ties at alphabetically by acronym).

def calc_distance(x0, y0, x1, y1):
    return sqrt(pow(x1-x0, 2) + pow(y1-y0, 2))

conn.create_function('calc_distance', 4, calc_distance)

# Predetermined values (feel free to change if you want)
folk_ID = 1
date = "2023-11-02 10:00:00"

command = f"""
SELECT * FROM (
    SELECT Places_Voting_Center.acronym FROM Places
    JOIN Places_Voting_Center ON Places.place_ID = Places_Voting_Center.place_ID
    JOIN Places_Operating_Periods ON Places_Voting_Center.op_ID = Places_Operating_Periods.op_ID
    JOIN Places_Coordinates ON Places.coordinates_ID = Places_Coordinates.coordinates_ID
    JOIN (
        SELECT Places_Coordinates.x_coord, Places_Coordinates.y_coord FROM Folks
        JOIN Places_Residences ON Folks.residence_ID = Places_Residences.place_ID
        JOIN Places ON Places.place_ID = Places_Residences.place_ID
        JOIN Places_Coordinates ON Places.coordinates_ID = Places_Coordinates.coordinates_ID
        WHERE Folks.personal_ID = {folk_ID}
    ) AS subquery
    WHERE '{date}' BETWEEN start_time AND end_time
    ORDER BY Places_Voting_Center.acronym
    LIMIT 1
)
"""

result = get_query(conn, c, command)



def find_closest_voting_center(folk_ID, date, data):
    if len(data) > 0:
        return data
    else:
        return None

conn.create_function('find_closest_voting_center', 3, find_closest_voting_center)

command = f"""
SELECT find_closest_voting_center({folk_ID}, '{date}', '{result[0][0]}')
"""

print_table(conn, c, ["Closest Voting Center Acronym"], command)

Closest Voting Center Acronym        
DEF                                  



In [23]:
# 10. For a given ballot, construct a cross-tabulation of voting centers (acronym) as rows, unique ballot answers (options) as columns, and cells indicating number of cast votes at the row's center with the column's option.

# Predetermined values (feel free to change if you want)
ballot_ID = 1

command = f"""
SELECT Places_Voting_Center.acronym, vote FROM Cast_Votes
JOIN Voting_Registry ON Cast_Votes.voting_reg_ID = Voting_Registry.voting_reg_ID
JOIN Places_Voting_Center ON Voting_Registry.voting_center_ID = Places_Voting_Center.place_ID
WHERE Cast_Votes.ballot_ID = {ballot_ID}
ORDER BY Places_Voting_Center.acronym, vote DESC
"""

query = get_query(conn, c, command)

tabulation = {}

# Populates tabulation
for result in query:
    if result[0] not in tabulation:
        tabulation[result[0]] = {
            'YES': 0,
            'NO': 0,
            'ABSTAIN': 0
        }
    
    if result[1] in tabulation[result[0]]:
        tabulation[result[0]][result[1]] += 1



# Creates temporary table
command = f"""
CREATE TABLE Temp_Tabulation (
    voting_center_acronym VARCHAR(4),
    YES_count INT,
    NO_count INT,
    ABSTAIN_count INT
)
"""

c.execute(command)
conn.commit()

# Populates data from the tabulation dictionary
for acronym in tabulation:
    command = f"""
    INSERT INTO Temp_Tabulation
    VALUES ('{acronym}', {tabulation[acronym]['YES']}, {tabulation[acronym]['NO']}, {tabulation[acronym]['ABSTAIN']})
    """

    c.execute(command)
    conn.commit()

command = f"""
SELECT * FROM Temp_Tabulation
"""

print_table(conn, c, ["Voting Center Acronym", "YES TOTAL", "NO TOTAL", "ABSTEIN TOTAL"], command)

# Drops the temporary table we made
command = """
DROP TABLE IF EXISTS Temp_Tabulation; 
"""

c.execute(command)
conn.commit()

Voting Center Acronym        YES TOTAL        NO TOTAL        ABSTEIN TOTAL        
ABC                          2                0               0                    
DEF                          1                5               0                    
GHI                          1                1               0                    



In [2]:
# Deletes all data from the database (if you want)

confirmation = str(input("Are you sure you want to delete the entire database? (type YES): "))

if confirmation == "YES":
    execute_SQL(conn, c, 'dropAll.sql')
else:
    print("Confirmation failed. Didn't delete the database.")

dropAll.sql Successful Executed


In [3]:
# Closes the connection

conn.close()
print("Connection Closed")

Connection Closed
